# Decision Trees & Random Forests Homework

В этом задании вам опять необходимо победить в конкурсе со своими однокурсниками. В этот раз разбиения на тест и трейн нет. Вам необходимо обучить свои модели, в срок прислать мне, я проверю ноутбуки, а потом зашлю ваши предикты на Kaggle в соревнование Titanic. Вы также можете делать посылки и смотреть, какие получаются результаты. Затем, по отношению вас друг к другу будет рассчитываться рейтинг.

Учтите, что метрика в этом соревновании `accuracy`. Использовать можно только решающие деревья и случайный лес.

Остальные условия в конце ноутбука.

Конец приема домашних заданий 15 октября 23:59.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
%matplotlib inline

Скачать данные по ссылке: https://www.kaggle.com/c/titanic/data
Если будут проблемы с сабмитом, то пишите в чат, но сначала изучите чужие кернелы.


In [2]:
data = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [3]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Место для feature engineering
В этой часте преобразуйте категориальные переменные, создавайте новые признаки.

Рекомендую изучить чужие кернелы на Каггл, чтобы создать какие-нибудь интересные фичи.

Для бейзлайна используются только числовые фичи. Это для его упрощения.
Но для решения домашнего задания лучше использовать и категориальные фичи, так как они могут помочь повысить скор.

Проведите с трейном и тестом одинаковые преобразования, но помните, что что к тесту применим только метод `.transform()`.

In [4]:
num_cols = [col for col in data.columns if data[col].dtype == 'float64' 
            or data[col].dtype == 'int64']
num_cols.remove('Survived')

# Заполним пропуски в данных
Заполним пропуски средним по колонкам. Но это не единственны вариант заполнения пропусков.

In [5]:
data = data.fillna(data.mean())
test = test.fillna(test.mean())

## Random Forests

Пример решения.

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import KFold 
from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import GridSearchCV
import time

/home/myltykritik/miniconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/home/myltykritik/miniconda3/lib/python3.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
cv = KFold(data.shape[0], shuffle=True, 
           random_state=42, n_folds=5)

In [8]:
gs = GridSearchCV(RandomForestClassifier(
    n_jobs=-1, # -1 значит, что используются все ядра компьютера
    random_state=42),
                  param_grid={'max_features': 
                              [None, 'log2', 'sqrt'], 
                              'max_depth': 
                              range(6, 10),
                              'n_estimators':
                              range(50, 150, 30),
                              'criterion': 
                              ['gini', 'entropy'],
                              'warm_start':
                              [True, False]},
                  cv=cv,
                  scoring='accuracy',
                  n_jobs=-1,
                  verbose=1)

In [9]:
start = time.time()
gs.fit(data[num_cols],  data['Survived'])
print((time.time() - start)/60)

Fitting 5 folds for each of 192 candidates, totalling 960 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   26.9s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 960 out of 960 | elapsed:  2.4min finished


2.454368547598521


In [10]:
gs.best_score_

0.7295173961840629

In [11]:
gs.best_params_

{'criterion': 'entropy',
 'max_depth': 6,
 'max_features': 'log2',
 'n_estimators': 50,
 'warm_start': True}

In [12]:
rfc = RandomForestClassifier(n_jobs=-1,
                             random_state=42,
                             n_estimators=gs.best_params_['n_estimators'], 
                             max_depth=gs.best_params_['max_depth'], 
                             max_features=gs.best_params_['max_features'],
                             criterion=gs.best_params_['criterion'],
#                              min_samples_leaf=gs.best_params_['min_samples_leaf'],
                             warm_start=gs.best_params_['warm_start'])
rfc.fit(data[num_cols], data['Survived'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=6, max_features='log2', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=True)

# Финальный результат
Здесь выведите модель, которая, как вы считаете, лучшая. Не нужно делать так, чтобы проверяющему приходилось пересчитывать гридсерч.

In [13]:
rfc = RandomForestClassifier(n_jobs=-1,
                             random_state=42,
                             n_estimators=50, 
                             max_depth=6, 
                             max_features='log2',
                             criterion='entropy',
                             warm_start=True)
rfc.fit(data[num_cols], data['Survived'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=6, max_features='log2', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=True)

# Создание сабмита

In [14]:
rfc_pred = rfc.predict(test[num_cols])
test['Survived'] = rfc_pred
test[['PassengerId','Survived']].to_csv('sabmit.csv', index=False)

# Домашнее задание

Бейзлайн очень простой: 0.64593 на лидерборде (название команды ECON2018).

Срок: до 15 октября 23:59.

Что нужно учесть:

1. Помните про переобучение, старайтесь не допускать его, чтобы на любом сабсете тестовых данных ваша модель показывала хорошие результаты. Используйте методы, предотвращающие переобучение.
2. Обновите библеотеки в окружении питона.
3. Использовать можно только решающие деревья и случайный лес.
2. В конце ноутбука обязательно построение итоговой модели без необходимости пересчитывать гридсерч.
3. Может оказаться, что максимизирование `roc_auc` в гридсерче принесет лучший скор на лидерборде.
4. Попробуйте кросс-валидацию.